In [1]:
!git clone https://github.com/Yang-Hyun-Jun/RLPortfolio-DDPG-COLAB.git

Cloning into 'RLPortfolio-DDPG-COLAB'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 99 (delta 50), reused 75 (delta 26), pack-reused 0
Unpacking objects: 100% (99/99), done.


In [2]:
!pip install mplfinance

     |████████████████████████████████| 70 kB 5.0 MB/s 


In [3]:
import sys
import os
import torch

In [4]:
sys.path.insert(0, "/content/RLPortfolio-DDPG-COLAB")

In [5]:
os.makedirs("/content/Metrics", exist_ok=True)
os.makedirs("/content/Models", exist_ok=True)

In [6]:
import DataManager
import utils

from Learner import DDPGLearner
from Test import DDPGTester

In [7]:
# Data Load

stock_list = ["AAPL", "JD", "WBA"]
path_list = []

for stock_name in stock_list:
  path = utils.SAVE_DIR + "/RLPortfolio-DDPG-COLAB/Data/" + stock_name
  path_list.append(path)

train_data, test_data = DataManager.get_data_tensor(path_list,
                                                    train_date_start="2014-06-02",
                                                    train_date_end="2019-12-31",
                                                    test_date_start="2020-01-02",
                                                    test_date_end="2021-12-31")

-------------------------------------
학습 데이터 시작 거래일: 20140602
학습 데이터 마지막 거래일: 20191231
테스트 데이터 시작 거래일: 20200102
테스트 데이터 마지막 거래일: 20211231
학습 데이터:(1407, 6, 3)
테스트 데이터:(505, 6, 3)
-------------------------------------


In [8]:
# Hyperparameters

K = len(stock_list)
lr = 1e-8
tau = 0.005
delta = 0.005
discount_factor = 0.9
num_episode = 1
balance = 15000000
batch_size = 30
memory_size = 100
min_trading_price = 0
max_trading_price = 500000

parameters1 = {"lr":lr, 
               "tau":tau,
               "K":K,
               "delta":delta,
               "chart_data":train_data,
               "discount_factor":discount_factor,
               "min_trading_price":min_trading_price,
               "max_trading_price":max_trading_price,
               "batch_size":batch_size,
               "memory_size":memory_size}

parameters2 = {"K":K,
               "balance":balance,
               "min_trading_price":min_trading_price,
               "max_trading_price":max_trading_price,
               "delta":delta,
               "test_data":test_data}

In [ ]:
learner = DDPGLearner(**parameters1)
learner.run(num_episode=num_episode, balance=balance)
learner.save_model(critic_path=utils.SAVE_DIR + "/Models" + "/DDPGPortfolio_critic.pth",
                   actor_path=utils.SAVE_DIR + "/Models" + "/DDPGPortfolio_actor.pth")

In [ ]:
tester = DDPGTester(**parameters2)
tester.run()